# Imports

In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
import numpy as np 
import pandas as pd 
import os

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display
from datetime import datetime as dt

In [ ]:
PATH = 'data/nyc-taxi/'
CSV_PATH = 'submissions/v09_ensemble_10trees_54M_rows/'

# Helper functions

In [ ]:
def split_df(df, y_fld, subset=None, random_state=42):
    
    if subset: df = df.sample(n=subset, random_state=random_state)

    y = df[y_fld].values
    df.drop(y_fld, axis=1, inplace=True)
    
    return df, y

# Load data

In [ ]:
seed = 42 # random seed for replication

In [ ]:
train_df = pd.read_feather('tmp/taxi-train-v4-chkpt5')
test_df = pd.read_feather('tmp/taxi-test-v4-chkpt5')
test_df_raw = pd.read_csv(f'{PATH}test.csv', usecols=['key'])

In [ ]:
train_df.shape, test_df.shape

In [ ]:
%%time
subsample = 50_000_000

# X_train, y_train = split_df(train_df, 'fare_amount', subset=subsample, random_state=seed)

X_train, y_train = split_df(train_df, 'fare_amount')

In [ ]:
X_train.shape, y_train.shape, test_df.shape

# Batch fit

In [ ]:
outdir = CSV_PATH
os.makedirs(outdir, exist_ok=True)
preds = []
n_forest=1

In [ ]:
%%time
for i in range (n_forest):
    
    new_seed = i
    
    print(f"Random Forest [{i+1}/{n_forest}] of 10 trees with random seed {new_seed}")
    m = RandomForestRegressor(n_estimators=10, min_samples_leaf=3, max_features=0.5, n_jobs=-1, random_state=new_seed)
    
    print("Fitting on training data...")
    m.fit(X_train, y_train)

    print("Predicting on test data...")
    test_y_predictions = m.predict(test_df)
    preds.append(test_y_predictions)      
    
    print("Creating submission file...")
    
    cur_dt = dt.now().strftime("%Y%m%d_%H%M%S")
    desc = 'v09_RF_' # description of submission for reference
    ensemb_desc = '_01x10_54M'
    subm_fn =  desc + cur_dt + ensemb_desc + '.csv'
    subm_path = f'{outdir}/{subm_fn}'

    # Write the predictions to a CSV file which we can submit to the competition.
    submission = pd.DataFrame(
        {'key': test_df_raw.key, 'fare_amount': test_y_predictions},
        columns = ['key', 'fare_amount'])
    submission.to_csv(subm_path, index = False)
    
    print(f"Submission: {subm_fn}")
    print()

In [ ]:
# preds[0]

In [ ]:
# test_y_predictions = np.mean(np.vstack([preds]),axis=0)

# Results